In [1]:
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import KFold
from sklearn import linear_model
import numpy as np
import gc 
from tqdm import tqdm
from sklearn.neighbors import KNeighborsRegressor

In [7]:
from sklearn.model_selection import train_test_split
data=pd.read_csv('https://raw.githubusercontent.com/Premalatha-success/Datasets/main/TaxiFare.csv')
train, test = train_test_split(data,train_size = 0.7, test_size = 0.3)

In [8]:
print(len(train))
train.head() 

35000


,unique_id,amount,date_time_of_pickup,longitude_of_pickup,latitude_of_pickup,longitude_of_dropoff,latitude_of_dropoff,no_of_passenger
46376,41:00.0,8.9,2012-03-08 22:41:00 UTC,-73.978322,40.753332,-73.952137,40.772193,2
31919,47:41.0,6.9,2011-01-09 01:47:41 UTC,-73.982824,40.745107,-73.965345,40.758388,1
29185,40:22.0,10.5,2010-11-13 13:40:22 UTC,-73.976524,40.751972,-73.992044,40.759918,2
15652,09:43.0,7.3,2011-09-27 16:09:43 UTC,-73.984900,40.737100,-73.996600,40.727000,2
7879,23:00.0,6.5,2014-05-09 09:23:00 UTC,-73.956327,40.781807,-73.965215,40.769612,3


In [18]:
X_test = test.iloc[:,2:]
Xt = train.iloc[:5,3:]
X_test = X_test[Xt.columns]
Xt.head()
X_test.head()
import gc
gc.collect()

1023

In [19]:
X_test.head()

,longitude_of_pickup,latitude_of_pickup,longitude_of_dropoff,latitude_of_dropoff,no_of_passenger
26363,-73.997140,40.737013,-73.975967,40.782192,1
40185,-73.993228,40.727821,-73.993717,40.747446,1
7039,-73.997292,40.741756,-74.002159,40.750736,1
3323,-73.966743,40.770968,-73.988043,40.764932,1
34275,-73.955858,40.782235,-73.958695,40.777464,1


In [20]:
len(train)

35000

In [63]:
knnregressoroutputs = []
knnregressoroutputstest = []
X_test2 = train.iloc[3500:,:]
X_test2 = X_test2[Xt.columns]
for i in tqdm(range(len(train)//1000)):
    neigh = KNeighborsRegressor(n_neighbors=2)
    df_chunk = train.iloc[i*10**3:(i+1)*10**3, :]
    X = df_chunk.iloc[:,3:]
    y = df_chunk['amount']
    neigh.fit(X,y)
    y_test = neigh.predict(X_test)
    y_test1 = neigh.predict(X_test2)
    knnregressoroutputs.append(y_test)
    knnregressoroutputstest.append(y_test1)
    gc.collect()

100%|██████████| 35/35 [00:11<00:00,  2.95it/s]


In [33]:
res = knnregressoroutputs[0]
for i in knnregressoroutputs[1:]:
    res+=i
res/=len(knnregressoroutputs)
holdout = pd.DataFrame({'unique_id': test.unique_id, 'amount': res})
holdout.to_csv('submission.csv', index=False)

res = knnregressoroutputstest[0]
for i in knnregressoroutputstest[1:]:
    res+=i
res/=len(knnregressoroutputstest)
holdout = pd.DataFrame({'fare_amount': res})
holdout.to_csv('test_result.csv', index=False)

In [60]:
import pickle
file_name='regressor.pkl'
f = open(file_name,'wb')
ks = []
errs = [] 
gc.collect() 
X_test = train.iloc[len(train)-1000:,3:] 
y_test_real = train.iloc[len(train)-1000:]["amount"] 
print(len(X_test))

1000


In [61]:
Xt = train.iloc[:5,3:] 
X_test = X_test[Xt.columns] 
Xt.head() 
X_test.head() 
gc.collect() 
for k in tqdm(range(2, 20)): 
  knnregressoroutputs = [] 
for i in tqdm(range(len(train)//1000000)): 
  neigh = KNeighborsRegressor(n_neighbors=k) 
  df_chunk = train.iloc[i10**6:i+110**6, :] 
  X = df_chunk.iloc[:,3:]

100%|██████████| 18/18 [00:00<00:00, 13460.06it/s]
0it [00:00, ?it/s]


In [62]:
import pickle
y = df_chunk['amount']
neigh.fit(X,y)
pickle.dump(neigh,f)
y_test = neigh.predict(X_test)
knnregressoroutputs.append(y_test)
gc.collect()
res = knnregressoroutputs[0]
for i in knnregressoroutputs[1:]:
  res+=i
res/=len(knnregressoroutputs)
ks.append(k)
errs.append(np.sqrt(((res-y_test_real) ** 2).mean()))
print(len(knnregressoroutputs))

1
